# Record Linkage

## Table of Contents

- [Initialization](#Initialization) 
- [Introduction](#Introduction)
- [Data Definition](#Data-Definition)
- [Parsing - with `split` method](#Parsing---with-split-method)
- [Regular Expressions - RE or regex](#Regular-Expressions---RE-or-regex)

    - [regex - Get Last Name](#regex---Get-Last-Name)
    - [regex - Get First Name](#regex---Get-First-Name)
    - [regex - Get Middle Name](#regex---Get-Middle-Name)
    - [_Exercise 1 - Regular Expressions_](#Exercise-1---Regular-Expressions)

- [String Comparators](#String-Comparators)
- [Fellegi-Sunter Record Linkage](#Fellegi-Sunter-Record-Linkage)

    - [_Exercise 2 - Value Comparison_](#Exercise-2---Value-Comparison)
    - [_Exercise 3 - Record Comparison_](#Exercise-3---Record-Comparison)

- [References & Further Readings](#References-&-Further-Readings)

## Initialization

- Back to the [Table of Contents](#Table-of-Contents)

Before we start the Record linkage assignment, we need to import the packages we will be using.  Please run the following code cell:

In [ ]:
# Importing the modules required in this workbook
import MySQLdb
import numpy
import pandas
import re
import jellyfish
import math

## Introduction

- Back to the [Table of Contents](#Table-of-Contents)

In this lesson we will learn a little about how to use Python for cleaning input data, including using regular expressions, as well as the basic idea behind probabilistic record linkage. These two topics fit together naturally, because data cleaning can have a signficant impact on the success of record linkage. Being able to compare fields that were normalized the same way will give better results.

We will use two datasets for the exercises in this chapter. The first will be a list of NIH projects and researchers pulled from the class database. The second is a list of university employees that was scraped from public web sites. 

## Data Definition

- Back to the [Table of Contents](#Table-of-Contents)

Before we begin the task of record linkage, it's important that we understand the variables in our data. In this workbook, we will take a cursory look at some of the values in our data and compute some simple statistics to ensure that the content makes sense. 

Begin by loading the two data sets into pandas data frames. After we load the two data sets, we all the `head` method on the first data set to examine the first few reords.

In [ ]:
# Make connections to the database
# Please use your own credentials
user = "jmorgan"
password = "mCw7iLdQyV@TGeGon3fB"
database = "homework"

# Invoke the connect() function, passing parameters in variables.
db = MySQLdb.connect( user = user, passwd = password, db = database )

# Output basic database connection info.
print(db)

# Load ucpay data directly through the DB connection
uc = pandas.read_sql('select * from UCPay2011;', con = db)

# Load nsf award data in 2010-2012 
nsf = pandas.read_sql('select * from NSF_Award;', con = db)

# Don't forget to close the connection
db.close()

In [ ]:
# Get the first few records from the UC employee file.
uc.head()

Here are some initial thoughts about the data:
* The ID field looks like a unique identifier that is specific to the data set. If we thought that we were going to use this identifier to link to other data from the same source, then we might take a loser look to see if the values should be interpreted as numbers or character strings. Ideally, this information would appear in the data documentation.
* We should check to make sure that the year field contains valid values.
* The campus and title fields look like they should be interpreted as finite categorical variables.
* The name field appears to contain some redacted values, probably due to a privacy agreement. We will want to link the subset of valid name fields.

Let's perform some quick summaries of the fields in the data.

To get a list of the unique values in a pandas column/Series, call the `.unique()` method on it - like `.value_counts()` from last assignment, only not sorted by frequency of use.

We'll ignore the numeric columns for now because we won't be using them for linkage, but a thorough data validation and definition process would ensure that these columns are valid and consistent as well.

In [ ]:
# print the distinct values in the year, campus, and title columns
print("Distinct years = ", uc["year"].unique())
print("Distinct campuses = ", uc["campus"].unique())
print("Distinct titles = ", uc["title"].unique())

# There are too many titles to display, so let's get the count
print("Number of distinct titles = ", len(uc["title"].unique()))

# Print the total number of rows and the number of rows with valid name values
print("Total rows = ", len(uc))
name = uc["name"]
print("Rows with valid names = ", len(name[name != "***********"]))


Next we'll take a look at the second data set.

In [ ]:
nsf.head()

Observations:
* There are separate fields for first name and last name
* The data include researchers from universities throughout the US, not just in the UC system
* For UC schools we should be able to create a field corresponding to the campus field in the UC data, but this will require some recoding.

In the next two sections we will address some of the data issues that we've identied. Before moving on, we will update the `uc` dataset so that it only contains records with valid name fields.

In [ ]:
# remove those records without a valid name
uc_name = uc[name != "***********"]
uc_name.head()

## Parsing - with `split` method

- Back to the [Table of Contents](#Table-of-Contents)

Above, we noted that we need to parse the name field in the UC data into first, middle, and last name fields. First, let's see what we can do with the built-in `split` method. By default, the `split` method returns a list of strings obtained by splitting the original string on spaces:

In [ ]:
print("Lorem Ipsum".split())
print("carrot cake".split())
print("Bogart, Humphrey".split())

By default, `split` treats whitespace as delimiting characters, and multiple consecutive whitespace characters are treated like a single delimiter. We can also pass an argument to `split` to be the delimiter. If we set the delimiter explicitly, then multiple delimiters are not combined.

In [ ]:
print("Hi,       Mom!".split())
print("Hi,       Mom".split(","))

# in the following example, the result will include the empty strings between the first and second commas
# and the second and third commas
print("Hi,,,Mom!".split(","))

We can map the `split` function over the values in the UC employee `name` column.  This uses `pandas` "`.apply()`" method to apply a function to all elements in a column.  To call a given function on a Series, pass its name to the "`.apply()`" function, including any package information needed (so in example below, we are telling it to call the "`split`" method from package "`str`"), not in quotation marks:

    # get "name" column.
    name_column_series = uc_name[ "name" ]

    # apply the split method to each value in the column.
    split_name_column_series = name_column_series.apply( str.split )

More on "`.apply()`": [http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.apply.html)

In [ ]:
uc_name["name"].apply(str.split).head()

This is pretty good! Fortunately for us, the UC data includes a space between the last name and the semicolumn separating it from the first name. This is unusual, but it means that the built-in `split` method is sufficient (we don't have to worry about parsing a semi-colon off each last name value).

In the code cell that follows, we define functions `GetLastName` and `GetFirstName` that will call the `split` method and extract the appropriate values from the resulting list. We won't worry about middle names, because the NSF data doesn't include them.

It turns out that all name fields in the `uc_name` data frame contain more than one word, even after filtering out the redacted names. Our `GetFirstName` function will have to perform a check to make sure that there is a first name to get.

_**NOTE** - the following code displays a warning (essentially warning us that we are creating a new copy of a column - which we are!) that we can safely ignore._

In [ ]:
# Simple functions that extract first name and last name
def GetFirstName(name):
    parts = name.split()
    
    if len(parts) >= 3:
        return parts[2]
    else:
        return None

def GetLastName(name):
    parts = name.split()
    return parts[0]

uc_name["FirstName"] = uc_name["name"].apply(GetFirstName)
uc_name["LastName"] = uc_name["name"].apply(GetLastName)

# Let's see the result
uc_name[["name", "FirstName", "LastName"]].head(15)

I intentionally showed 15 rows instead of the default 10 so that we could see an example of our simple approach failing. The second to last row in the result contains a last name that comprises several words. Our rule, which extracted the first word to be the "last name" does not give the correct result.

There is a more subtle problem as well. It's likely that the third name, WOO YONG, should not be split into a first name and middle name component. That is, WOO YONG may in fact be the first name of the referenced individual. This format is common for names from some countries, including China and Korea.

Let's redefine the `GetFirstName` and `GetLastName` functions so that the last name consists of everything before the semicolumn and the first name consists of the first word after the semicolumn. This doesn't solve our problem with Chinese and Korean names, but it's a good start.

_**NOTE** - the following code also displays a warning (essentially warning us that we are creating a new copy of a column - which we are!) that we can safely ignore._

In [ ]:
# Improved functions that extract first name and last name
def GetFirstName(name):
    parts = name.split(";") 
    
    if len(parts) > 1:
        # The name contains a semicolumn. Take the part after the semicolumn, split on whitespace,
        # and grab the first word
        return parts[1].split()[0]
    else:
        return None

def GetLastName(name):
    parts = name.split(";") 
    
    # Return the first part (everything up to the first semicolumn)
    # call the `strip` method to remove the space between the last name and the semicolumn
    return parts[0].strip()

uc_name["FirstName"] = uc_name["name"].apply(GetFirstName)
uc_name["LastName"] = uc_name["name"].apply(GetLastName)

# Let's see the result
uc_name[["name", "FirstName", "LastName"]].head(15)

## Regular Expressions - RE or regex

- Back to the [Table of Contents](#Table-of-Contents)

We were able to solve our parsing problem using the `split` method. Sometimes we need a more heavy duty way to search and extract text. In this section we introduce the basics of regular expressions, because they are a common approach to text parsing and a powerful one, too.

Regular expressions are a mini-language for searching text. We have already noted that regular expressions are *powerful*, but they can also be *complex*. 

> Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems. 

> &mdash; Jamie Zawinski

### regex - Get Last Name

- Back to the [Table of Contents](#Table-of-Contents)

In the previous section we were trying to extract first and last names from the UC name field:

In [ ]:
uc_name["name"].head()

We used basic python functions to try to split the names into first and last name components. In this section we'll use regular expressions to accomplish this task.

When defining a regular expression search pattern, it is a good idea to start out by writing down, explicitly, in plain English, what you are trying to search for and exactly how you identify when you've found a match.

For example, if we look at an author field formatted as "`<last_name> ; <first_name> <middle_name>`", in plain English, this is how I would explain where to find the last name: "starting from the beginning of the line, take all the characters until you see a semi-colon."

We can build a regular expression that captures this idea from the following components:

* `^` Matches beginning of the line
* `.` Matches any character
* `+` A modifier that means "match one or more of the preceding expression"

In a regular expression, there are special reserved characters and character classes like those in the list above.  Anything that is not a special character or class is just looked for explicitly (for example, a semi-colon is not a special character in regular expressions, so if it is in a regular expression pattern, the regular expression processor will just be looking for a semi-colon in the string, at that point in the pattern).

This results in the regular expression:

    ^.+;
    
We start at the beginning of the line ( "^" ), matching any characters ( ".+" ) until we come to the literal character of a semi-colon ( ";" ).

In python, to use a regular expression like this to search for matches in a given string, we use the built-in "`re`" package ( [https://docs.python.org/2/library/re.html](https://docs.python.org/2/library/re.html) ), specifically the "`re.search()`" method.  To use "`re.search()`", pass it first the regular expression you want to use to search, enclosed in quotation marks, and then the string you want to search within:

In [ ]:
last_name_match = re.search(r"^.+;", "Su ; Yu-Wen")
last_name_match

Two things:
* We used "raw" string syntax `r""` to prevent python from escaping any special characters that might appear in our regular expression.
* If the regular expression matches something in the searched string, then `m` holds a python `MatchObject` instance ( [https://docs.python.org/2/library/re.html#re.MatchObject](https://docs.python.org/2/library/re.html#re.MatchObject) ) or a child class of that object. Otherwise, it will be `None`.

To see the matched substring, use the `group` function.

In [ ]:
last_name_match.group()

This is almost what we want, but not quite.  We need the semi-colon in our regular expression in order to know where the last name ends, but it would be nice if we could omit the semi-colon from the value we find.  The solution is to use regular expression "groups".  Within a regular expression, one creates a "group" by putting parentheses around a part of the regular expression that one wants to refer to later.  You can have mulitple "groups" in a single regular expression.

* `(...)` Creates a group that can be referred to later.

So, in our regular expression, we'll place any characters up to a comma in parentheses: 

In [ ]:
# run regular expression
last_name_match = re.search(r"^(.+);", "Franklin ; Benjamin")

# get full match text
full_match_text = last_name_match.group()

# get contents of first group defined in regex.
first_group_match_text = last_name_match.group( 1 )

# print results
print( "- Full match text: \"" + full_match_text + "\"" )
print( "- First group match text: \"" + first_group_match_text + "\"" )

In a match-data object like `last_name_match`:

- `last_name_match.group()` is a synonym for `last_name_match.group(0)`.  Either of these refers to what was matched by the entire regular expression, not just the matches contained in an individual group.
- `last_name_match.group(1)` will show you what was matched by the expression between the first set of parentheses.
- `last_name_match.group(2)` will show you what was matched by the expression between the second set of parentheses, if a second group was defined.
- ... - and so on, for as many groups as you define.

### regex - Get First Name

- Back to the [Table of Contents](#Table-of-Contents)

Now let's write a regular expression that will match the first name of the author (that is, the first word after the semi-colon). In addition to the bits of regular expressions we saw above, here are some other useful character classes:

* `\w` Matches a single alphanumeric character
* `\W` Matches anything that is not an alphanumeric character
* `\s` Matches a single whitespace character
* `\S` Matches anything that is not a whitespace character.
* `*` Matches 0 or more instances of the expresson that precedes the asterisk.

In [ ]:
regex = r";\s*(\w+)"
m = re.search(regex, "Eisenhower ; Dwight David")
m.group(1)

The above regular expression:

    ;\s*(\w+)
    
matches as follows

- first finds a semi-colon - "`;`".
- then, after the semi-colon, the "`\s*`" looks for from 0 to as many as are embedded white space characters immediately following the semi-colon character.
- the "`(\w+)`" group definition then matches any 1 or more alphanumeric charaters - "`\w+`", and stores that set of characters in a group.

Regular expressions are a powerful tool and we're barely scratching the surface. (See - [References & Further Reading](#References-&-Further-Readings) for more information)

### regex - Get Middle Name

- Back to the [Table of Contents](#Table-of-Contents)

Now is the time to practise what we have learned (parsing strings and regular expression in Python). As we already have walked through the example of extracting first name and last name, we should be able to get middle name as well. The following is an example how we can do this by using python parsing functions.

In [ ]:
# In this cell, you will see the example function using split and join to obtain middle names
# Note that the middle names appear after comma and come after first name if they exist.

def GetMiddleName_split(name):
    '''
    name: given a person's full name
    '''
    # extract the middle name based on its position in the full name
    # we can use "join" to unlist in python.

    parts = name.split(";")
    
    if len(parts[1]) > 1:
        # The name contains a semicolumn. Take the part after the semicolumn, split on whitespace,
        # and grab starting from the second word

        # extract the middle name based on its position in the full name
        # We can use "join" to unlist in python.
        if " ".join(parts[1].split()[1:]):
            return " ".join(parts[1].split()[1:])
        else:
            return None

    else:
        return None

### Exercise 1 - Regular Expressions

- Back to the [Table of Contents](#Table-of-Contents)

In the exercise, you will be asked to complete a function that uses regular expressions to parse the middle name out of an author string.

Notes on this exercise:

- for this exercise, we define a "middle name" as the part of a name string that:

    - is to the right of the semi-colon
    - AND to the right of the first name
    - AND to the right of any whitespace that follows the first name
    
- It is a good idea to test your regular expression before you run it in your function.  You can test a regular expression online at [https://regex101.com/#python](https://regex101.com/#python).  Enter your regular expression string, without surrounding quotation marks, in the "REGULAR EXPRESSION" field at the top of the page, then enter a test name string in the "TEST STRING" box just below it.
- You can (and should!) use one or more groups - () - to target just the middle name within your larger regular expression.

In [ ]:
# In this cell, you need to define your own function using regular expression to obtain middle names
# Note that the middle names appear after semicolumn and come after first name and a whitespace.
# Please be careful when you define your own regular expression.
# You can use either group() or groups() to get middle names in the last step.


def GetMiddleName_re(name):
    
    '''
    name: given a person's full name
    '''
    
    ### BEGIN SOLUTION
    # Define the regular expression to get the middle name
    regex = r";\s*\w+\s(\w+)"
    # Store the re.search outcome
    pattern = re.search(regex, name)
    # We can use groups() to get the matched middle names
    # If Null, the function should return None.
    if pattern:
        return pattern.groups()[0]
    else:
        return None
    ### END SOLUTION
    
#-- END function GetMiddleName_re() --#

Below, we've provided two code cells to help you assess how well your "`GetMiddleName_re()`" function compares to the middle name finder defined above ( "`GetMiddleName_split()`" ).

In the first cell, we populate columns/Series with the results of each method for each row:

- 'MiddleName' contains result of "`GetMiddleName_split()`"
- 'MiddleName_re' contains result of "`GetMiddleName_re()`"
- BONUS - 'MiddleNameMatch' contains a 1 if the value of 'MiddleName' is equal to the value in 'MiddleName_re', and a 0 if not.

We then display the first 30 rows, so you can visually compare the middle names found by the two different methods.

_**NOTE** - the following code also displays warnings that we can safely ignore._

In [ ]:
# First, Let's see how your functions work

# place split-based middle name in "MiddleName"
uc_name["MiddleName"] = uc_name["name"].apply(GetMiddleName_split)

# place regular expression-based middle name in "MiddleName_re"
uc_name["MiddleName_re"] = uc_name["name"].apply(GetMiddleName_re)

# create column "MiddleNameMatch" where value is 1 of two middle names match, 0 if not.
uc_name[ "MiddleNameMatch" ] = numpy.where( uc_name["MiddleName"].isin( uc_name["MiddleName_re"] ), 1, 0 )

# output first 30 columns.
uc_name[ [ "name", "FirstName", "LastName","MiddleName","MiddleName_re", "MiddleNameMatch" ] ].head( 30 )

In the second, we then divide the sum of the values in the 'MiddleNameMatch' column (the count of matches) by the total number of rows and multiply the result by 100 to get percentage agreement.

Your goal should be to get 80% agreement or better (for as simple as it is to parse name parts for a person, this is a relatively tough problem in computer science).

In [ ]:
# TEST - calculate percent agreement to see how well the two columns match.

# get sum of new column
match_count = uc_name[ "MiddleNameMatch" ].sum()

# get length of DataFrame
row_count = len( uc_name )

# calculate percentage agreement
from __future__ import division
percent_agree = ( match_count / row_count ) * 100

print( "Percent agreement = " + str( percent_agree ) )
assert percent_agree > 80

## String Comparators

- Back to the [Table of Contents](#Table-of-Contents)

In this section we will demonstrate different string comparison algorithms provided by the [jellyfish](https://github.com/sunlightlabs/jellyfish) package ( [https://github.com/sunlightlabs/jellyfish](https://github.com/sunlightlabs/jellyfish) ).

For each method we examine, we'll write a function that accepts a name that we want to find matches for in the NSF data and that returns a list of the names in the NSF data that are most similar to the name of interest.

We will start by creating a `set` of unique first names from the NSF data. The `FirstName` field is missing some values which are represented as NaN in the data frame. To prevent errors later on, we only include valid character strings (which have type `str`) in our list of unique names.

In [ ]:
# If we were starting from scratch, we'd need to import jellyfish
# import jellyfish

# Make a set storing the unique first name with respect to the nsf dataset
unique_first_names = set( name for name in nsf[ "FirstName" ] if type( name ) == str )

Next, we will define our function "`closest_names`" that:

* Accepts a string name we are interested in matching with NSF names as input argument "`name_IN`".
* Accepts an optional number of results we want returned as input argument "`result_count_IN`".
* Compares the name in `name_IN` to each name in `unique_first_names` and calculates the "distance" between the two strings using the Levenshtein Distance string comparator from `jellyfish`.
* Return a list of size `result_count_IN` of names in `uniq_first_names` that are "closest" to `name_IN`.

From wikipedia, the Levenshtein Distance is defined as:

> "In information theory and computer science, the Levenshtein distance is a string metric for measuring the difference between two sequences. Informally, the Levenshtein distance between two words is the minimum number of single-character edits (i.e. insertions, deletions or substitutions) required to change one word into the other. It is named after Vladimir Levenshtein, who considered this distance in 1965."

> &mdash; [https://en.wikipedia.org/wiki/Levenshtein_distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

_Note that in the comparison we capitalize both names being compared, so that letter case doesn't affect the final distance._

In [ ]:
def closest_names( name_IN, result_count_IN = 10 ):

    # return reference
    results_OUT = []
    
    # declare variables
    other_name = ""
    cleaned_name = ""
    cleaned_other_name = ""
    get_distance_lambda = None
    
    # first, standardize the name - convert to upper case and to unicode.
    cleaned_name = name_IN.upper()
    cleaned_name = unicode( cleaned_name )
    
    # First create a list of tuples (other_name, distance), where other_name is taken from uniq_first_names
    distances = []

    # loop over unique_first_names to calculate and store distances
    for other_name in unique_first_names:
        
        # standardize the other name.
        cleaned_other_name = other_name.upper()
        cleaned_other_name = unicode( cleaned_other_name )
        
        # get distance from name to other_name (converted to upper case so we are case-insensitive.)
        distance_value = jellyfish.levenshtein_distance( cleaned_name, cleaned_other_name )
        
        # add tuple to distances
        current_tuple = ( other_name, distance_value )
        distances.append( current_tuple )
    
    #-- END loop over unique_first_names --#
    
    # Sort distances by the second element in the tuple.
    
    # define lambda function to retrieve the distance (the second item in the tuple)
    #    and return it.  Lambda functions are little one line functions.  More information:
    #    https://docs.python.org/2/reference/expressions.html#lambda
    get_distance_lambda = lambda distance_tuple_list_IN : distance_tuple_list_IN[ 1 ]
    
    # sort matching names by distance
    results_OUT = sorted( distances, key = get_distance_lambda )
    
    # get the number of results requested using Python's slice notation.
    results_OUT = results_OUT[ : result_count_IN ]
    
    # return results
    return results_OUT
    
    '''
    # For reference, compacted version - you can do this, but please don't.
    
    # First create a list of tuples (other_name, distance), where other_name is taken from uniq_first_names
    distances = [ ( other_name, jellyfish.levenshtein_distance( unicode( name_IN.upper() ), unicode( other_name.upper() ) ) )
        for other_name in unique_first_names ]

    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1])[:result_count_IN]
    '''

#-- END function closest_names() --#

Let's try it out on some names.

In [ ]:
# Experiment the function with several names.
print( closest_names( "Jennifer" ) )
print( closest_names( "Sonya" ) )
print( closest_names( "Wai Tong" ) )

Recall that Levenshtein distance is a kind of edit distance. Edit distances count the number of edit operations needed to change one word to another, and different edit distances count different edit operations as valid. In the case of Levenshtein distance, the valid edit operations are inserting a letter, deleting a letter, or changing a letter. 

It would be interesting to compare this output to the output from other string comparators included in the jellyfish package:

* **`jellyfish.lenvenstein_distance`** - _Levenshtein distance_: edit distance where the valid operations are inserting a letter, deleting a letter, or changing a letter
* **`jellyfish.damerau_levenshtein_distance`** - _Levenshtein-Damerau distance_: edit distance which includes the same operations as Levenshtein distance but also allows transposing two adjacent letters. This can be useful for finding words with typos.
* **`jellyfish.jaro_winkler`** - _Jaro-Winkler distance_: a fast-to-compute string distance based on common letters between two words

_Note: For edit distance smaller numbers indicate closer strings, but for Jaro-Winkler distance larger values indicate closer strings._

Let's update our `closest_names` function so that we can specify the string comparator we want to use.  Changes from previous function:

- add ability to pass in the string distance calculation function you want to use as an argument, named "`string_comparator_function_IN`".

    - just pass the name of the function, not in quotation marks, and not followed by parentheses (just like they are shown in the list of functions above).

- add ability to reverse sort order for returning "closest" strings to name passed in.  New parameter "`reverse_sort_IN`" defaults to `False` (to match distance scores where a larger number indicates two strings being further apart).  Set it to `True` for distance scores like Jaro-Winkler distance where a larger number indicates two strings are closer together.

In [ ]:
def closest_names_2( string_comparator_function_IN, name_IN, reverse_sort_IN = False, result_count_IN = 10 ):

    # return reference
    results_OUT = []
    
    # declare variables
    other_name = ""
    cleaned_name = ""
    cleaned_other_name = ""
    get_distance_lambda = None
    
    # first, standardize the name - convert to upper case and to unicode.
    cleaned_name = name_IN.upper()
    cleaned_name = unicode( cleaned_name )
    
    # First create a list of tuples (other_name, distance), where other_name is taken from uniq_first_names
    distances = []

    # loop over unique_first_names to calculate and store distances
    for other_name in unique_first_names:
        
        # standardize the other name.
        cleaned_other_name = other_name.upper()
        cleaned_other_name = unicode( cleaned_other_name )
        
        # get distance from name to other_name (converted to upper case so we are case-insensitive.)
        distance_value = string_comparator_function_IN( cleaned_name, cleaned_other_name )
        
        # add tuple to distances
        current_tuple = ( other_name, distance_value )
        distances.append( current_tuple )
    
    #-- END loop over unique_first_names --#
    
    # Sort distances by the second element in the tuple.
    
    # define lambda function to retrieve the distance (the second item in the tuple)
    #    and return it.  Lambda functions are little one line functions.  More information:
    #    https://docs.python.org/2/reference/expressions.html#lambda
    get_distance_lambda = lambda distance_tuple_list_IN : distance_tuple_list_IN[ 1 ]
    
    # sort matching names by distance
    results_OUT = sorted( distances, key = get_distance_lambda, reverse = reverse_sort_IN )
    
    # get the number of results requested using Python's slice notation.
    results_OUT = results_OUT[ : result_count_IN ]
    
    # return results
    return results_OUT
    
    '''
    # For reference, compacted version - you can do this, but please don't.
    
    # First create a list of tuples (other_name, distance), where other_name is taken from uniq_first_names
    distances = [(other_name, string_comparator(unicode(name.upper()), unicode(other_name.upper())))
                 for other_name in uniq_first_names]
    
    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1], reverse=reverse_sort)[:n]
    '''

#-- END function closest_names_2() --#

In [ ]:
# Try it!
print( "Closest names for \"William\" using Levenshtein-Damerau distance:" )
print( closest_names_2( jellyfish.damerau_levenshtein_distance, "William" ) )

print( "\n\nClosest names for \"William\" using Levenshtein-Damerau distance:" )
print( closest_names_2( jellyfish.jaro_winkler, "Wiliam", reverse_sort_IN = True ) )

## Fellegi-Sunter Record Linkage

- Back to the [Table of Contents](#Table-of-Contents)

Fellegi-Sunter Record Linkage is a probabilistic method that uses comparisons of fields that contain the same substantive types of data between records to calculate a weighted probability that records in different data sets refer to the same entity.  Examples of the types of data items that might be compared in this method include gender, date of birth, age, and parts of a name.

In this section we will "manually" perform the steps in Fellegi-Sunter record linkage. Our goal is to illustrate the Fellegi-Sunter algorithm by breaking it into bitesize pieces. 

In our example we will compare first names and last names using Jaro-Winkler distance. In the Fellegi-Sunter algorithm, the result of a field comparison is assumed to follow a multinomial distribution. That means it can only take on finitely many values. Therefore we will define a function that compares two strings and returns the value 2, 1, or 0 to indicate an exact match, a nearly exact match, or anything else. 

### Exercise 2 - Value Comparison

- Back to the [Table of Contents](#Table-of-Contents)

In exercise 2, we will implement the value comparison stage of Fellegi-Sunter Record Linkage.  You will implement a function named "`fuzzy-string-comparator`" that accepts two strings and returns one of the following match levels:

- 2 - exact match
- 1 - close match
- 0 - not a match

To assess whether the two strings passed in match, we'll convert both strings to capital letters, decode them into unicode, then calculate the Jaro-Winkler distance between the two strings.  We'll then assign a match level based on where the resulting match score falls in the following ranges:

- 2 - exact match - score greater than or equal to ( >= ) 0.92
- 1 - close match - score less than 0.92 but greater than or equal to 0.85.
- 0 - not a match - score less than 0.85.

Finally, we'll return that match score.

In [ ]:
# Please complete the following function that tells us how different two input strings are.
# It returns a match level with value 2, 1 or 0 (larger value means higher similarity)
# Calculate Jaro-Winkler distance after converting two strings into capital characters.
# Please use these three criteria, >=0.92, >=0.85, <0.85, to determine match level.

def fuzzy_string_comparator( string_1_IN, string_2_IN ):

    '''
    string_1_IN : input string No.1
    string_2_IN : input string No.2
    '''
    
    # return reference
    match_level_OUT = -1

    # Check if they are all strings
    if ( ( type( string_1_IN ) != str ) or ( type( string_2_IN ) != str ) ):
        
        match_level_OUT = 0
    
    #-- END check to see if strings are actually strings. --#
    
    ### BEGIN SOLUTION
    # declare variables
    cleaned_string_1 = ""
    cleaned_string_2 = ""
    distance = -1
    
    # convert strings to upper case, then to unicode
    
    # string 1
    cleaned_string_1 = string_1_IN.upper()
    cleaned_string_1 = unicode( cleaned_string_1 )

    # string 2
    cleaned_string_2 = string_2_IN.upper()
    cleaned_string_2 = unicode( cleaned_string_2 )
    
    # Calculate Jaro-Winkler distance after converting two strings into capital characters.
    distance = jellyfish.jaro_winkler( cleaned_string_1, cleaned_string_2 )

    # According to different thresholds, return the match level
    if distance >= 0.92:

        match_level_OUT = 2

    elif distance >= 0.85:
    
        match_level_OUT = 1

    else:
    
        match_level_OUT = 0
        
    #-- END conditional to set match level. --#

    ### END SOLUTION

    return match_level_OUT

#-- END function fuzzy_string_comparitor --#

In [ ]:
# Let's see how the fuzzy_string_comparator works
score1 = fuzzy_string_comparator( "joshua", "joshua" )
score2 = fuzzy_string_comparator( "joshua", "joshau" )
score3 = fuzzy_string_comparator( "joshua", "todd" )

print( "Match level for joshua-joshua: " + str( score1 ) )
print( "Match level for joshua-joshau: " + str( score2 ) )
print( "Match level for joshua-todd: " + str( score3 ) )

# tests for our grading program:
assert score1 == 2
assert score2 == 2
assert score3 == 0

The above function compares *text fields* in a record (other types of data would need different means of comparison). Next, we define a function that compares *records*.  This record comparison function assumes that records will have the form of a tuple: (identifier, first name, last name).  It returns a length 2 tuple that gives the result of applying our fuzzy string comparator to the first name and to the last name.

In [ ]:
# Comparison_vector compare a pair of records, which consists of first name and last name.
# It returns a tuple with 2 match levels.

def compare_records( record_1_IN, record_2_IN ):

    '''
    record_1_IN : input record No.1
    record_2_IN : input record No.2
    '''
    
    # return reference
    results_OUT = None
    
    # declare variables
    field_1_match_level = -1
    field_2_match_level = -1
    
    # record_1_IN and record_2_IN have the form (id, first name, last name)
    
    # m, n store the comparing outcomes of first name and last name.
    field_1_match_level = fuzzy_string_comparator( record_1_IN[ 1 ], record_2_IN[ 1 ] )
    field_2_match_level = fuzzy_string_comparator( record_1_IN[ 2 ], record_2_IN[ 2 ] )

    results_OUT = ( field_1_match_level, field_2_match_level )

    return results_OUT

#-- END function compare_records() --#

In [ ]:
# Try out the compare_records function

print( compare_records( ( 1, "joshua", "tokle" ), ( 2, "joshua", "smith") ) )
print( compare_records( ( 3, "joshua", "tokle" ), ( 4, "josue", "tolke") ) )

### Exercise 3 - Record Comparison

- Back to the [Table of Contents](#Table-of-Contents)

Next, we'll work on implementing the section of Fellegi-Sunter Record Linkage that calculates a weighted probability that two records from different data sets refer to the same entity.

Fellegi-Sunter Record Linkage uses two different sets of probabilities per pair of data items as weights in this step: m-weights and u-weights.

For a given pair of data items that represent the same conceptual thing, for each match level:
- An **m-weight** is the probability of seeing a particular match level if we assume that we are comparing two records that represent the same individual.
- A **u-weight** is the probability of seeing a particular match level if we assume that that we are comparing two records that do *not* represent the same individual.

For example, thinking of probabilities when two records are the same (m-weights), if two records represent the same person, the first names and last names should match with high probability - match level 2.  So, the m-weight for first name and last name having a match level of 2 when two records refer to the same person should be large.

On the other hand, in the context of probabilities when two records are different (u-weights), suppose we had month of birth in our data set.  The probability that two random individuals will have the same month of birth is about 1/12, so for records that are not the same person, we would assign a u-weight of about 1/12 to the birth year being identical (where for a field like social security number, the u-weight of two different people having the same social security number is 0).

Let's assign some preliminary and arbitrary m- and u-weights for first name and last name.

- first name

    - m-weights:

        - match level 0: **_0.01_** (very unlikely the same person will have different first names)
        - match level 1: **_0.14_** (also pretty unlikely that first names for a person will be mostly the same)
        - match level 2: **_0.85_** (very likely that a person's first names will match exactly)

    - u-weights:

        - match level 0: **_0.88_** (probability that different people will have different first names)
        - match level 1: **_0.10_** (probability that different people's first names will be mostly the same)
        - match level 2: **_0.02_** (probability that different people will have same first name)
- last name

    - m-weights:

        - match level 0: **_0.01_** (very unlikely the same person will have different last names)
        - match level 1: **_0.09_** (also pretty unlikely that last names for a person will be mostly the same)
        - match level 2: **_0.90_** (very likely that a person's last names will match exactly)

    - u-weights:

        - match level 0: **_0.91_** (probability that different people will have different first names)
        - match level 1: **_0.08_** (probability that different people's first names will be mostly the same)
        - match level 2: **_0.01_** (probability that different people will have same first name
    
In practice you would likely start with guesses or very general estimates like these, but then try to better estimate these by trying to fit them to a model or at least tweaking them after seeing preliminary output. In this case, we are just going to run with our initial estimates so we can move efficiently through this process.

In the cell below, we create dictionaries that contain the m- and u-weights for our two columns.

In [ ]:
# Make dictionaries to hold m_weights and u-weights.  In each dictionary, the weights for
#    a given field are mapped to a string label for that field ("first_name" and "last_name").
# Weights are captured in lists of length 3, with the index in the list matching each of the 
#   match levels that can be returned by the fuzzy string comparator.
# In this list, we go from match level 0 (not the same), to match level 2 (identical)
#    as we move from left to right in the list, with each position in the list holding the
#    corresponding weight for that match level.

m_weights_dict = {}
m_weights_dict[ "first_name" ] = [ 0.01, 0.14, 0.85 ]  # m-weights corresponding to first name
m_weights_dict[ "last_name" ] = [ 0.01, 0.09, 0.90 ] # m-weights corresponding to last name

u_weights_dict = {}
u_weights_dict[ "first_name" ] = [ 0.88, 0.10, 0.02 ] # u-weights corresponding to first name
u_weights_dict[ "last_name" ] = [ 0.91, 0.08, 0.01 ] # u-weights corresponding to last name

Now, for exercise 3, we are going to define a function that uses these weights to compare two records and return their record-level match score.

In Fellegi-Sunter Record Linkage the match score for a given record is the at calculates a weighted probability that two records from different data sets refer to the same entity.



In [ ]:
def match_score( record_1_IN, record_2_IN ):

    '''
    record_1_IN: input record No.1
    record_2_IN: input record No.2
    '''
    
    # return reference
    score_OUT = -1
    
    # declare variables
    match_level_tuple = None
    match_level_first_name = -1
    match_level_last_name = -1
    
    # Calulate the similarity level using compare_records
    match_level_tuple = compare_records( record_1_IN, record_2_IN )
    match_level_first_name = match_level_tuple[ 0 ]
    match_level_last_name = match_level_tuple[ 1 ]
    
    # Use match levels and m- and u-weights to calculate a match score for this record.

    ### BEGIN SOLUTION
    
    # declare variables
    m_weights_list_first_name = None
    m_weights_list_last_name = None
    u_weights_list_first_name = None
    u_weights_list_last_name = None
    first_name_m_weight = -1
    first_name_u_weight = -1
    last_name_m_weight = -1
    last_name_u_weight = -1
    log_prob_given_match = None
    log_prob_given_nonmatch = None
    
    # get lists of m- and u- weights for each field from weights dictionaries defined above.
    m_weights_list_first_name = m_weights_dict[ "first_name" ]
    m_weights_list_last_name = m_weights_dict[ "last_name" ]
    u_weights_list_first_name = u_weights_dict[ "first_name" ]
    u_weights_list_last_name = u_weights_dict[ "last_name" ]
   
    # get weights for match levels returned by compare_records.
    first_name_m_weight = m_weights_list_first_name[ match_level_first_name ]
    first_name_u_weight = u_weights_list_first_name[ match_level_first_name ]
    last_name_m_weight = m_weights_list_last_name[ match_level_last_name ]
    last_name_u_weight = u_weights_list_last_name[ match_level_last_name ]
    
    # calculate log-probabilities for each field assuming a match (m-weight),
    #    and assuming a non-match (u-weight).  Log-probability is the natural
    #    log (math.log() in Python) of the probability of a given match level.
    log_prob_first_name_given_match = math.log( first_name_m_weight )
    log_prob_last_name_given_match = math.log( last_name_m_weight )
    log_prob_first_name_given_no_match = math.log( first_name_u_weight )
    log_prob_last_name_given_no_match = math.log( last_name_u_weight )
    
    # For match and no-match, sum the log-probabilities for each field.
    
    # What's the log-probability of seeing this comparison vector if the records are a match?
    log_prob_given_match = log_prob_first_name_given_match + log_prob_last_name_given_match
    
    # What's the log-probability of seeing this comparison vector if the records are a nonmatch?
    log_prob_given_nonmatch = log_prob_first_name_given_no_match + log_prob_last_name_given_no_match
    
    # match score is the sum of the log probabilities given a match
    #    minus the sum of the log probabilites given no match.
    score_OUT = log_prob_given_match - log_prob_given_nonmatch
    
    ### END SOLUTION
    
    # return match score.
    return score_OUT

#-- END function match_score() --#

In [ ]:
# Have a rough look at its sample output
print(match_score((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(match_score((1, "joshua", "tokle"), (4, "joshu", "tolke")))
print(match_score((1, "joshua", "tokle"), (7, "christina", "jones")))

Finally, we can try to link UC employees to NSF researchers. We will apply **blocking** to reduce the number of comparisons we perform. In particular, I am only going to compare graduate students whose last name begins with the letter "H". I will also limit the NSF researchers to those located in California.

In [ ]:
# Subset the uc data, taking those last names start with "H"
last_name_startswith_h = uc_name["LastName"].apply(lambda s: s[0] == "H")
uc_h = uc_name[startswith_h]

# http://stackoverflow.com/questions/17957890/pandas-select-from-dataframe-using-startswith#comment26265963_17958424

# Take Californian nsf records for comparison, also we only look at last names starting with "H"
nsf_ca = nsf[(nsf['StateCode'] == "CA")]
startswith_h = nsf_ca["LastName"].apply(lambda s: type(s) == str and s[0].upper() == "H")
nsf_h = nsf_ca[startswith_h]

In [ ]:
# This cell takes a minute to execute

# Create an empty list to save the outputs
potential_matches = []

# Loop over uc data frame by lines
for _, uc_row in uc_h.iterrows():
    # Store ID, FirstName and LastName of uc
    rec1 = (uc_row["ID"], uc_row["FirstName"], uc_row["LastName"])
    # Loop over nsf data frame by lines
    for nsf_ix, nsf_row in nsf_h.iterrows():    
        # Store ID, FirstName and LastName of nsf
        rec2 = (nsf_ix, nsf_row["FirstName"], nsf_row["LastName"])
        # Calculate the match score of each pair of records
        score = match_score(rec1, rec2)
        # Save those pairs with score equal to or greater than 0.5
        if score >= 0.5:
            potential_matches.append((score, rec1, rec2))
            
# Sort the output so the best matches appear at the top
potential_matches = sorted(potential_matches, key=lambda x: x[0], reverse=True)
potential_matches

# How did we do?

## References & Further Readings

- Back to the [Table of Contents](#Table-of-Contents)

### Parsing

* Python online ducumentation: https://docs.python.org/2/library/string.html#deprecated-string-functions
* Python 2.7 Tutorial(Splitting and Joining Strings): http://www.pitt.edu/~naraehan/python2/split_join.html

### Regular Expression

* Python documentation: https://docs.python.org/2/library/re.html#regular-expression-syntax
* Online regular expression tester (good for learning): http://regex101.com/

### String Comparators

* GitHub page of jellyfish: https://github.com/jamesturk/jellyfish
* Different distances that measure the differences between strings:
    - Levenshtein distance: https://en.wikipedia.org/wiki/Levenshtein_distance
    - Damerau–Levenshtein distance: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
    - Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    - Hamming distance: https://en.wikipedia.org/wiki/Hamming_distance
    - Match rating approach: https://en.wikipedia.org/wiki/Match_rating_approach

### Fellegi-Sunter Record Linkage 

* Introduction to Probabilistic Record Linkage: http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf
* Paper Review: https://www.cs.umd.edu/class/spring2012/cmsc828L/Papers/HerzogEtWires10.pdf

